In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import seaborn as sns
from statsmodels import api as sm
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
import os
import numpy as np
import random
from scipy.stats.mstats import gmean

In [2]:
def make_data(x, str,w):
    x=pd.DataFrame(x)
    window=w
    for i in range(1, window):
        column=f'{str}_lag{i}'
        x[column]=x[str].shift(i)
    
#     for i in range(-1, -window, -1):
#         column=f'{str}_lag{i}'
#         x[column]=x[str].shift(i)   
    return x

In [3]:
class Bagging:
    def __init__(self, n_models=10, ratio=1.0, model=NearestNeighbors, model_params={}):
        self.n_models=n_models
        self.ratio=ratio
        self.model=model
        self.model_params=model_params
        self.models=[]
        
    def fit(self, x):
        self.models=[]
        n_sample=int(len(x)*self.ratio)
        for _ in range(self.n_models):
            index=random.sample(list(range(len(x))), n_sample)
            model=self.model(**self.model_params)
            model.fit(x[index])
            self.models.append(model)
        return self
    
    def predict(self, x):
        ds=[model.kneighbors(x)[0] for model in self.models]
        d=[np.mean(di, axis=1) for di in ds]
        mx=[np.max(z) for z in d]
        anorm=[distance/max for distance, max in zip(d, mx)]
        anorm=np.array(anorm)
#         return anorm.mean(axis=0)
        return gmean(anorm, axis = 0)

    

In [4]:
if __name__ == '__main__':
    df = pd.read_csv( os.path.join('csv_label', 'learning_data_10.csv') )
    df=df[df['day']<=2]
    period=3
    for i in set(df['day']):
        df.loc[df['day']==i, 'trend']=df.loc[df['day']==i, 'original'].rolling(window=period+1, min_periods=1).mean()
    df=df.dropna().reset_index(drop=True)
    calc_mean=100
    for i in set(df['day']):
        df.loc[df['day']==i, 'mean']=df.loc[df['day']==i, 'original'].rolling(window=calc_mean+1, min_periods=1).mean()
    df=df.dropna().reset_index(drop=True)

    lag=[] 
    window=10
    for str in ['original', 'trend']:
        lag.append(make_data(df[str], str, window))

    lag.append(df['mean'])
    lag.append(df['day'])
    lag.append(df['label'])
    df_add_lag=pd.concat(lag, axis=1)
    df=df_add_lag.dropna().reset_index(drop=True)
    display('all_data:', df)
    

    train=df[(df['label']==0)&(df['day']==0)][df.columns[10:-2]].values
    test=df[(df['day']!=0)][df.columns[10:-2]].values
    
    n_neighbors=1
    plf=Bagging(n_models=30, ratio=0.1, model=NearestNeighbors, 
                model_params={'n_neighbors':2})
    plf.fit(train)
    d=plf.predict(test)

    test_for_plot=df[(df['day']!=0)]['original'].values[window-1:]
    test_for_plot

    label=df[(df['day']!=0)][df.columns[-1]].values[window-1:]*100
    label_index=range(len(label))

    for i in range(16):
        fig = plt.figure(figsize=(35,10))
        ax1 = fig.add_subplot(111)
        # ax2 = ax1.twinx()

        ax1.plot(d*100+0, '-r',linewidth = 1 )
        ax1.plot(test_for_plot, '-k',linewidth = 2)
        ax1.fill_between(label_index, label, facecolor='lime' )
        plt.xlim(i*17999,17999*(i+1))
        plt.ylim(20, 90)
        plt.show()

'all_data:'

,original,original_lag1,original_lag2,original_lag3,original_lag4,original_lag5,original_lag6,original_lag7,original_lag8,original_lag9,...,trend_lag3,trend_lag4,trend_lag5,trend_lag6,trend_lag7,trend_lag8,trend_lag9,mean,day,label
0,55.7,55.5,55.7,54.7,54.0,53.6,53.2,52.8,52.8,53.4,...,53.875,53.400,53.100,53.050,53.000,53.100,53.400,54.140000,0,0
1,55.6,55.7,55.5,55.7,54.7,54.0,53.6,53.2,52.8,52.8,...,54.500,53.875,53.400,53.100,53.050,53.000,53.100,54.272727,0,0
2,55.8,55.6,55.7,55.5,55.7,54.7,54.0,53.6,53.2,52.8,...,54.975,54.500,53.875,53.400,53.100,53.050,53.000,54.400000,0,0
3,55.7,55.8,55.6,55.7,55.5,55.7,54.7,54.0,53.6,53.2,...,55.400,54.975,54.500,53.875,53.400,53.100,53.050,54.500000,0,0
4,55.0,55.7,55.8,55.6,55.7,55.5,55.7,54.7,54.0,53.6,...,55.625,55.400,54.975,54.500,53.875,53.400,53.100,54.535714,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151441,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,56.4,56.0,...,55.850,56.250,56.450,56.400,56.325,56.175,56.300,50.328713,2,0
1151442,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,56.4,...,55.525,55.850,56.250,56.450,56.400,56.325,56.175,50.398020,2,0
1151443,53.1,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,...,55.050,55.525,55.850,56.250,56.450,56.400,56.325,50.464356,2,0
1151444,53.0,53.1,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,...,54.550,55.050,55.525,55.850,56.250,56.450,56.400,50.533663,2,0
